## Import library

In [6]:
import warnings
warnings.filterwarnings('ignore')

import sys

MAIN_PATH = "/home/trduong/Data/interpretable_machine_learning/Source Code/my_work"


sys.path.insert(
    1,
    MAIN_PATH + '/lib'
)

sys.path.insert(
    1,
    MAIN_PATH + '/config'
)

import pandas as pd
import numpy as np

import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor

# EconML
from econml.dml import LinearDMLCateEstimator, ForestDMLCateEstimator
from econml.cate_interpreter import SingleTreeCateInterpreter, SingleTreePolicyInterpreter

import numpy as np
import pandas as pd
import logging
import dowhy.datasets
import dowhy
import incremental_ps_score_estimator as ipse
import math
import timeit
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import PolynomialFeatures

from econml.drlearner import ForestDRLearner
from econml.drlearner import LinearDRLearner
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from econml.metalearners import SLearner
from econml.metalearners import XLearner
from econml.metalearners import TLearner
from econml.drlearner import LinearDRLearner

from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from econml.dml import ForestDMLCateEstimator
from dowhy import CausalModel
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import sem
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from math import sqrt
from econml.dml import LinearDMLCateEstimator
from econml.inference import BootstrapInference
from econml.dml import SparseLinearDMLCateEstimator
from sklearn.linear_model import LassoCV, ElasticNetCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

## Load data

In [7]:
file_path = "https://msalicedatapublic.blob.core.windows.net/datasets/Pricing/pricing_sample.csv"
df = pd.read_csv(file_path)

In [8]:
df.head()

,account_age,age,avg_hours,days_visited,friends_count,has_membership,is_US,songs_purchased,income,price,demand
0,3,53,1.834234,2,8,1,1,4.903237,0.960863,1.0,3.917117
1,5,54,7.171411,7,9,0,1,3.330161,0.732487,1.0,11.585706
2,3,33,5.351920,6,9,0,1,3.036203,1.130937,1.0,24.675960
3,2,34,6.723551,0,8,0,1,7.911926,0.929197,1.0,6.361776
4,4,30,2.448247,5,8,1,0,7.148967,0.533527,0.8,12.624123


In [9]:
df.price.value_counts()

1.0    4346
0.8    3089
0.9    2565
Name: price, dtype: int64

In [10]:
### change to binary treatment
# df = df[df.price != 0.8]
df['price'] = df['price'].apply(lambda x: 0.85 if x != 1.0 else x)
df['price_bin'] = df['price'].apply(lambda x: 1 if x == 1.0 else 0)

In [11]:
df.price.value_counts()

0.85    5654
1.00    4346
Name: price, dtype: int64

In [12]:
# Define estimator inputs
Y = df["demand"]  # outcome of interest
T = df["price"]  # intervention, or treatment
T_bin = df['price_bin']
X = df[["income"]]  # features
W = df.drop(columns=["demand", "price", "income"])  # confounders


In [13]:
# Get test data
X_test = np.linspace(0, 5, 100).reshape(-1, 1)
X_test_data = pd.DataFrame(X_test, columns=["income"])

## Build the treatment model

In [9]:
model_t = LogisticRegression()
model_t.fit(W, T_bin)

LogisticRegression()

In [10]:
df['p1'] = 1 - model_t.predict_proba(W)[:,:1]
df['p0'] = model_t.predict_proba(W)[:,:1]

## Calculate profit

In [11]:
def gamma_fn(X):
    return -3 - 14 * (X["income"] < 1)

def beta_fn(X):
    return 20 + 0.5 * (X["avg_hours"]) + 5 * (X["days_visited"] > 4)

def demand_fn(data, T):
    Y = gamma_fn(data) * T + beta_fn(data)
    return Y

def true_te(x, n, stats):
    if x < 1:
        subdata = df[df["income"] < 1].sample(n=n, replace=True)
    else:
        subdata = df[df["income"] >= 1].sample(n=n, replace=True)
    te_array = subdata["price"] * gamma_fn(subdata) / (subdata["demand"])
    if stats == "mean":
        return np.mean(te_array)
    elif stats == "median":
        return np.median(te_array)
    elif isinstance(stats, int):
        return np.percentile(te_array, stats)
    
def revenue_fn(data, discount_level1, discount_level2, baseline_T, policy):
    policy_price = baseline_T * (1 - discount_level1) * policy + baseline_T * (1 - discount_level2) * (1 - policy)
    demand = demand_fn(data, policy_price)
    rev = demand * policy_price
    return rev

In [12]:
import math
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [13]:

def incre_ps(delta, data):
    discount_level1 = 0.1
    discount_level2 = 0
    q1 = (delta * data['p1']) / (delta * data['p1'] + data['p0'])
    policy_price = 1 * (1 - discount_level1) * q1 + 1 * (1 - discount_level2) * (1 - q1)
    demand = policy_price + beta_fn(data)
    return tf.reduce_mean(demand)


def optimization(data, threhold, true_effect):
    delta = tf.Variable(3., trainable = True)
    for i in tqdm(range(40000)):
        with tf.GradientTape() as tape:
            influence = incre_ps(delta, data)
            loss = tf.math.abs(true_effect - influence)
            d_delta = tape.gradient(loss, delta)
            opt = tf.keras.optimizers.Adam(learning_rate=0.001)
            opt.apply_gradients(zip([d_delta], [delta]))
    return delta

In [15]:
optimization(df, 0.01, true_effect)

## Explore revenue with continuous delta

In [16]:
df.head()

,account_age,age,avg_hours,days_visited,friends_count,has_membership,is_US,songs_purchased,income,price,demand,price_bin,p1,p0
0,3,53,1.834234,2,8,1,1,4.903237,0.960863,1.00,3.917117,1,0.997154,0.002846
1,5,54,7.171411,7,9,0,1,3.330161,0.732487,1.00,11.585706,1,0.997198,0.002802
2,3,33,5.351920,6,9,0,1,3.036203,1.130937,1.00,24.675960,1,0.997334,0.002666
3,2,34,6.723551,0,8,0,1,7.911926,0.929197,1.00,6.361776,1,0.997131,0.002869
4,4,30,2.448247,5,8,1,0,7.148967,0.533527,0.85,12.624123,0,0.002164,0.997836


In [19]:
policy = np.where(df['price'] == 1, 1, 0)
rev = revenue_fn(data = df, 
           discount_level1 = 0, 
           discount_level2 = 0.15, 
           baseline_T = 1, policy = policy)
print("Revenue {}".format(np.mean(rev)))

Revenue 14.298001236178342


In [24]:
from collections import Counter
delta_seq = np.random.normal(0.1, 30, 1000)
rev_seq = []
dictionary_ = []
delta_result = []
cov_seq = []
for delta in tqdm(delta_seq):
    df['q1'] = (delta * df['p1']) / (delta * df['p1'] + df['p0'])
    policy = np.where(df['q1'] >= 0.5, 1, 0)
    rev = revenue_fn(data = df, 
           discount_level1 = 0, 
           discount_level2 = 0.15, 
           baseline_T = 1, policy = policy)
    print(Counter(policy))
    print("Revenue {}".format(np.mean(rev)))


  4%|▍         | 39/1000 [00:00<00:05, 184.58it/s]

Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0

  8%|▊         | 79/1000 [00:00<00:04, 189.64it/s]

Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0

 12%|█▏        | 119/1000 [00:00<00:04, 193.30it/s]

...

Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0

 99%|█████████▉| 993/1000 [00:04<00:00, 208.45it/s]

Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0

100%|██████████| 1000/1000 [00:04<00:00, 203.06it/s]

Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
Counter({0: 5654, 1: 4346})
Revenue 14.298001236178342
